In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
import torch
from torch import nn, optim
import torch.nn.functional as F
import tqdm
from tqdm import tnrange
#use_cuda = torch.cuda.is_available()
device = torch.device("cuda")

In [2]:
train=pd.read_csv("train_total.csv",index_col=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65628 entries, 0 to 65627
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   Unnamed: 0                    65628 non-null  int64
 1   countryName                   65628 non-null  int64
 2   eprtrSectorName               65628 non-null  int64
 3   EPRTRAnnexIMainActivityLabel  65628 non-null  int64
 4   FacilityInspireID             65628 non-null  int64
 5   facilityName                  65628 non-null  int64
 6   City                          65628 non-null  int64
 7   targetRelease                 65628 non-null  int64
 8   pollutant                     65628 non-null  int64
 9   reportingYear                 65628 non-null  int64
 10  MONTH                         65628 non-null  int64
 11  DAY                           65628 non-null  int64
 12  CONTINENT                     65628 non-null  int64
 13  max_wind_speed                6

In [3]:
pca_3=['countryName','EPRTRAnnexIMainActivityLabel','FacilityInspireID','City',
       'MONTH','max_wind_speed', 'avg_wind_speed', 
       'min_wind_speed','max_temp','avg_temp', 'min_temp','DAY WITH FOGS', 'REPORTER NAME']
x=train[pca_3]#.values
y=train['pollutant']#.values#.reshape((65628,1))
x.shape

(65628, 13)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0,test_size=0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, random_state=0,test_size=0.5)

In [5]:
f=10 # no. of batches #15

train_batch = np.array_split(x_train, f) 
label_batch = np.array_split(y_train, f) 

val_batch = np.array_split(x_val, f)
val_label_batch = np.array_split(y_val, f)

test_batch = np.array_split(x_test, f)
test_label_batch = np.array_split(y_test, f)


for i in range(len(train_batch)):
    train_batch[i] = torch.from_numpy(train_batch[i].values).float()
for i in range(len(label_batch)):
    label_batch[i] = torch.from_numpy(label_batch[i].values).float().view(-1, 1)

for i in range(len(val_batch)):
    val_batch[i] = torch.from_numpy(val_batch[i].values).float()
for i in range(len(val_label_batch)):
    val_label_batch[i] = torch.from_numpy(val_label_batch[i].values).float().view(-1, 1)
    
for i in range(len(test_batch)):
    test_batch[i] = torch.from_numpy(test_batch[i].values).float()
for i in range(len(test_label_batch)):
    test_label_batch[i] = torch.from_numpy(test_label_batch[i].values).float().view(-1, 1)
    
    

    
print("Batch size:", len(train_batch[0]))

Batch size: 4594


In [8]:
class Regressor(nn.Module):#87%
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(13, 15)
        self.fc2 = nn.Linear(15, 17)
        self.fc3 = nn.Linear(17, 19)
        self.fc4 = nn.Linear(19, 25)
        self.fc5 = nn.Linear(25, 25)
        self.fc6 = nn.Linear(25, 25)
        self.fc7 = nn.Linear(25, 3)
        self.fc8 = nn.Linear(3, 1)
       
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))

        return x
    
model = Regressor()
train_losses, val_losses = [], []
criterion = nn.MSELoss()
optimizer = optim.Adamax(model.parameters(), lr=0.01) # 0.015 87
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.01, patience=15) 
total_epochs=0

In [9]:
epochs = 400
total_epochs+=epochs

for e in tnrange(epochs):
    
    model.train()
    train_loss = 0
    val_loss_1=0
    val_loss_sum = 0
    
    for i in range(len(train_batch)):
        
        optimizer.zero_grad()
        output = model(train_batch[i])
        loss = criterion(output, label_batch[i])
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
        with torch.no_grad():
            
            for j in range(len(val_batch)):
                
                val_output = model(val_batch[j])
                val_loss =  criterion(val_output, val_label_batch[j])
                val_loss_1+=val_loss.item()
        val_loss_sum=val_loss_1/len(val_batch)
        
        
    print("Epoch :", e, "train_loss :", train_loss/len(train_batch), "Val loss: ", val_loss_sum/len(val_batch))    
    val_losses.append(val_loss_sum/len(val_batch))    
    train_losses.append(train_loss/len(train_batch))   

C:\Users\ivanl\AppData\Local\Temp/ipykernel_7804/3967188832.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for e in tnrange(epochs):


  0%|          | 0/400 [00:00<?, ?it/s]

Epoch : 0 train_loss : 37.86270986795425 Val loss:  1.8093427968025206
Epoch : 1 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 2 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 3 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 4 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 5 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 6 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 7 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 8 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 9 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 10 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 11 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 12 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Epoch : 13 train_loss : 1.847102439403534 Val loss:  1.8093427896499634
Ep

KeyboardInterrupt: 

In [ ]:

frm=10 # does not 
plt.figure(figsize=(25,15))
plt.plot(train_losses[frm:], label='Training loss')
plt.plot(val_losses[frm:], label='Validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()
plt.legend()
#ax = plt.gca()
#ax.set_ylim([0, 2])

In [ ]:
correct=0
i=0
y_pred=[]
with torch.no_grad():
    for batch in test_batch :
        for j in range(len(batch)):
            x = model(batch[j])
            #print(round(x.item()))
            y_pred.append(round(x.item()))
#print(y_pred)

print(classification_report(y_test, y_pred))